In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install numpy
!pip install tensorflow
!pip install matplotlib

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import mixed_precision


In [4]:
# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

In [5]:

# Path of train and test data
train_dir = "/content/drive/MyDrive/klimb_llm_optimization_challenge/seg_train"
test_dir = "/content/drive/MyDrive/klimb_llm_optimization_challenge/seg_test"

# Data configs
batch_size = 16
img_height = 150
img_width = 150

In [6]:
# Load a subset of train data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
train_ds = train_ds.take(100)  # Load only 100 batches for training


Found 14033 files belonging to 6 classes.
Using 11227 files for training.


In [7]:
# Load a subset of test data
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
test_ds = test_ds.take(20)  # Load only 20 batches for evaluation


Found 2998 files belonging to 6 classes.
Using 599 files for validation.


In [8]:
# Training the MASTER Model - using Transfer Learning
# Here we are using a smaller base model
base_model = keras.applications.ResNet50(
    weights='imagenet',
    input_shape=(img_height, img_width, 3),
    include_top=False)

base_model.trainable = False

inputs = keras.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(6)(x)

master_model = keras.Model(inputs, outputs)

# Use a faster optimizer
optimizer = keras.optimizers.RMSprop()

master_model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

epochs = 10
early_stop = EarlyStopping(monitor='val_loss', patience=5)

master_model.fit(train_ds, epochs=epochs, callbacks=[early_stop])


94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.6912 - sparse_categorical_accuracy: 0.7600

100/100 [==============================] - 396s 3s/step - loss: 0.6912 - sparse_categorical_accuracy: 0.7600
Epoch 2/10
 99/100 [============================>.] - ETA: 0s - loss: 0.3148 - sparse_categorical_accuracy: 0.8807

100/100 [==============================] - 6s 55ms/step - loss: 0.3160 - sparse_categorical_accuracy: 0.8800
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.2383 - sparse_categorical_accuracy: 0.9131

100/100 [==============================] - 6s 55ms/step - loss: 0.2383 - sparse_categorical_accuracy: 0.9131
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 0.1889 - sparse_categorical_accuracy: 0.9306

100/100 [==============================] - 6s 58ms/step - loss: 0.1889 - sparse_categorical_accuracy: 0.9306
Epoch 5/10
 99/100 [============================>.] - ETA: 0s - loss: 0.1561 - sparse_categorical_accuracy: 0.9444

...

100/100 [==============================] - 6s 54ms/step - loss: 0.1272 - sparse_categorical_accuracy: 0.9525
Epoch 7/10
100/100 [==============================] - ETA: 0s - loss: 0.1020 - sparse_categorical_accuracy: 0.9669

100/100 [==============================] - 7s 65ms/step - loss: 0.1020 - sparse_categorical_accuracy: 0.9669
Epoch 8/10
100/100 [==============================] - ETA: 0s - loss: 0.0951 - sparse_categorical_accuracy: 0.9681

100/100 [==============================] - 6s 54ms/step - loss: 0.0951 - sparse_categorical_accuracy: 0.9681
Epoch 9/10
 99/100 [============================>.] - ETA: 0s - loss: 0.0751 - sparse_categorical_accuracy: 0.9792

100/100 [==============================] - 6s 53ms/step - loss: 0.0762 - sparse_categorical_accuracy: 0.9781
Epoch 10/10
 99/100 [============================>.] - ETA: 0s - loss: 0.0698 - sparse_categorical_accuracy: 0.9811

100/100 [==============================] - 6s 60ms/step - loss: 0.0702 - sparse_categorical_accuracy: 0.9806


In [9]:
# Generate results on test data
results = master_model.evaluate(test_ds)
print(f"Test accuracy with trained master model: {results[1] * 100:.2f}%")

20/20 [==============================] - 105s 4s/step - loss: 0.3529 - sparse_categorical_accuracy: 0.8844
Test accuracy with trained master model: 88.44%


# Student model


In [42]:
import tensorflow as tf
from tensorflow.keras import layers, models

def train_student_model(base_model, train_ds, test_ds, img_height, img_width):
    # Define the student model architecture
    student_inputs = layers.Input(shape=(img_height, img_width, 3))
    x = base_model(student_inputs, training=False)  # Use the base model as a layer
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    student_outputs = layers.Dense(6)(x)
    student_model = models.Model(student_inputs, student_outputs)

    # Compile the student model
    student_model.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=[keras.metrics.SparseCategoricalAccuracy()])

    # Train the student model using the dataset
    student_model.fit(train_ds, epochs=10, validation_data=test_ds)

    return student_model

In [43]:
# Train the student model
student_model = train_student_model(base_model, train_ds, test_ds, 150, 150)

Epoch 1/10
702/702 [==============================] - 85s 105ms/step - loss: 0.8466 - sparse_categorical_accuracy: 0.8469 - val_loss: 0.3898 - val_sparse_categorical_accuracy: 0.8564
Epoch 2/10
702/702 [==============================] - 48s 68ms/step - loss: 0.2377 - sparse_categorical_accuracy: 0.9227 - val_loss: 0.3461 - val_sparse_categorical_accuracy: 0.9015
Epoch 3/10
702/702 [==============================] - 46s 66ms/step - loss: 0.1569 - sparse_categorical_accuracy: 0.9453 - val_loss: 0.4136 - val_sparse_categorical_accuracy: 0.8848
Epoch 4/10
702/702 [==============================] - 47s 66ms/step - loss: 0.1233 - sparse_categorical_accuracy: 0.9590 - val_loss: 0.5488 - val_sparse_categorical_accuracy: 0.8815
Epoch 5/10
702/702 [==============================] - 191s 272ms/step - loss: 0.0917 - sparse_categorical_accuracy: 0.9690 - val_loss: 0.5550 - val_sparse_categorical_accuracy: 0.8881
Epoch 6/10
702/702 [==============================] - 50s 70ms/step - loss: 0.0807 - sp

In [50]:
# Evaluate and compare the master and student models
print(f"Model size: {student_model.count_params() * 4 / (1024 ** 2):.2f} MB")
print(f"Model size: {master_model.count_params() * 4 / (1024 ** 2):.2f} MB")
print(f"Model size ratio: {student_model.count_params() / master_model.count_params():.2f}")

Master model size: 247.67 MB
Student model size: 90.03 MB
Model size ratio: 2.75


In [51]:
# Calculate latency (time taken for a single prediction)
import time

start_time = time.time()
_ = base_model.predict(test_ds.take(1))
master_latency = time.time() - start_time
print(f"Master model latency: {master_latency * 1000:.2f} ms")

start_time = time.time()
_ = student_model.predict(test_ds.take(1))
student_latency = time.time() - start_time
print(f"Student model latency: {student_latency * 1000:.2f} ms")

1/1 [==============================] - 0s 489ms/step
Master model latency: 677.52 ms
1/1 [==============================] - 0s 405ms/step
Student model latency: 649.15 ms


In [47]:
# Load a sample image for testing
sample_image = np.random.rand(1, img_height, img_width, 3)  # Single random image


In [48]:
# Perform inference on the sample image using the student_model
predictions = student_model.predict(sample_image)
predicted_class = np.argmax(predictions)

print(f"Predicted class: {predicted_class}")

1/1 [==============================] - 4s 4s/step
Predicted class: 4
